<a href="https://colab.research.google.com/github/sinjy1203/gender_classifier/blob/master/gender_classifier_full_data_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras import Model, Input
from tensorflow.keras import layers

In [0]:
from tensorflow.keras.applications import InceptionV3
conv_base = InceptionV3()
conv_base.summary()

In [0]:
from tensorflow.keras.utils import plot_model
plot_model(conv_base, to_file='model.png', show_shapes=True)

In [0]:
def inception(input):
    features = input.shape[-1] // 8
    branch_1 = layers.Conv2D(features*4, 1, padding='SAME', kernel_initializer="he_normal")(input)
    branch_1 = layers.BatchNormalization()(branch_1)
    branch_1 = layers.Activation("relu")(branch_1)
    branch_2 = layers.AveragePooling2D(3, 1, padding='SAME')(input)
    branch_2 = layers.Conv2D(features*2, 3, padding='SAME', kernel_initializer='he_normal')(branch_2)
    branch_2 = layers.BatchNormalization()(branch_2)
    branch_2 = layers.Activation("relu")(branch_2)
    branch_3 = layers.Conv2D(features*4, 1, padding='SAME', kernel_initializer="he_normal")(input)
    branch_3 = layers.BatchNormalization()(branch_3)
    branch_3 = layers.Activation("relu")(branch_3)
    branch_3 = layers.Conv2D(features*4, 3, padding='SAME', kernel_initializer="he_normal")(branch_3)
    branch_3 = layers.BatchNormalization()(branch_3)
    branch_3 = layers.Activation("relu")(branch_3)
    branch_4 = layers.Conv2D(features*6, 1, padding='SAME', kernel_initializer="he_normal")(input)
    branch_4 = layers.BatchNormalization()(branch_4)
    branch_4 = layers.Activation("relu")(branch_4)
    branch_4 = layers.Conv2D(features*6, 3, padding='SAME', kernel_initializer="he_normal")(branch_4)
    branch_4 = layers.BatchNormalization()(branch_4)
    branch_4 = layers.Activation("relu")(branch_4)
    branch_4 = layers.Conv2D(features*6, 3, padding='SAME', kernel_initializer="he_normal")(branch_4)
    branch_4 = layers.BatchNormalization()(branch_4)
    branch_4 = layers.Activation("relu")(branch_4)
    output = layers.concatenate([branch_1, branch_2, branch_3, branch_4], axis=-1)
    return output

input = Input(shape=(150, 150, 3))
x = layers.Conv2D(32, 3, padding='SAME', kernel_initializer='he_normal')(input)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Conv2D(64, 3, padding='SAME', kernel_initializer='he_normal')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Conv2D(128, 3, padding='SAME', kernel_initializer='he_normal')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPooling2D(2, strides=2)(x)
x = inception(x)
x = layers.MaxPooling2D(2, strides=2)(x)
x = inception(x)
x = layers.MaxPooling2D(2, strides=2)(x)
x = inception(x)
x = layers.MaxPooling2D(2, strides=2)(x)
x = inception(x)
x = layers.AveragePooling2D(9)(x)
output = layers.Dense(1, activation='sigmoid')(x)
model = Model(inputs=input, outputs=output)
# model.summary()

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
train_dir = "/content/drive/My Drive/image_data/data/train1"
val_dir = "/content/drive/My Drive/image_data/data/val/val_"

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150,150), batch_size=64, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150,150), batch_size=64, class_mode='binary')
val_generator = test_datagen.flow_from_directory(val_dir, target_size=(150,150), batch_size=64, class_mode='binary')

In [0]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(), metrics=['acc'])
history = model.fit_generator(train_generator, steps_per_epoch=162770 // 64, epochs=5, validation_data=val_generator, validation_steps=19867 // 64)
model.save("/content/drive/My Drive/model.h5")

870